In [1]:
import json # used for managing the JSON files from API
from urllib.request import urlopen # fetch URl of API
from pprint import pprint # just for printing values for human use
from copy import deepcopy

In [166]:
# Loads the papers to json from text file

with open("./stored_authors/papers.txt") as papers_file:
    papers = eval(papers_file.read())

In [3]:
authors_ids = {}
with open("./stored_authors/authors_ids.txt") as authors_file:
    authors_ids = eval(authors_file.read())
# pprint(authors_ids)

In [4]:
pprint(papers[0])

{'DOI': '10.1002/adem.200500223',
 'authors': ['75dc757852ded186f04bf13eae66c53803db5c82',
             '25e2798b6f1c8d5ca03eda52e347d33da845514f',
             '95f1bf7d79db23a55c6d8c155ecb3a6c2ec533ca',
             '7653e3bbcd863a9e7e7e142c07df15292a347874',
             '979826cedb64012e6ac7dd2787fcea904501fddc',
             '8e693fc4e5de76ff86bafd4577f7cd7a8556f74c'],
 'journal': 'Advanced Engineering Materials',
 'title': 'Improved Hydrogen Storage in the Metal-Organic Framework Cu3(BTC)2'}


In [5]:
# Just for visual clarity, not used for processing

papers_with_names = {}
for paper in papers:
    papers_with_names[ paper["DOI"] ] = []
    for person in paper["authors"]:
        papers_with_names[ paper["DOI"] ].append(authors_ids[person])
pprint(papers_with_names["10.1002/adem.200500223"])

[['P.', '', 'Krawiec'],
 ['Markus', '', 'Kramer'],
 ['M.', '', 'Sabo'],
 ['R.', '', 'Kunschke'],
 ['Heidrun', '', 'Fröde'],
 ['Stefan', '', 'Kaskel']]


In [6]:
duplicate_authors = {}
with open("./stored_authors/duplicate_authors.txt") as dup_file:
    duplicate_authors = eval(dup_file.read())

In [7]:
unknown_authors = {}
with open("./stored_authors/unknown_authors.txt") as unk_file:
    unknown_authors = eval(unk_file.read())

In [170]:
# not sure about this. Generate a coauthors dict? Only for unknown authors for right now

coauthors = {}

# Iterates through papers 
for paper in papers:
    
    # Iterates through authors
    for person in paper["authors"]:
        if person in unknown_authors or person in unknown_authors.values():
            
            # There is some problem here, I don't know what exactly though
            if not person in coauthors:
                coauthors[person] = []
            coauthors[person].extend(paper["authors"])
            coauthors[person].remove(person)
for person, coauthor_list in coauthors.items():
#     pprint(sorted(coauthor_list))
    coauthor_list = set(coauthor_list)
#     pprint(coauthor_list)
pprint(coauthors)

{'0000-0002-6496-8411': ['52f8e13f8f60b729c721ccd62e13a1452bf261b9',
                         '7c695798e9f88d40cb97ad74dffccd9a9ed123ec',
                         '2b02d2753c61258852ffb5000a973d0527bd2b1e',
                         '7c4f3bcf9c5624277042f7d15d3a1321b4c92107',
                         'aaf0129620e874e947eb1436a39e55ecfc72c4b5',
                         '6b01deba07049ca237abf78ec968305af6f217c5',
                         'e995a9ec8d260fa063a244c50acdea719c81164f',
                         'd48d73bd7e6593bf903b43f54d40bc2321cbe45d'],
 '00ae0596cc30940fcc470ce7acfd18f8fcede4ef': ['4ed693d8163b226fa4ea2695dae71a31dd958582',
                                              'b9592982bb587e609f2cf4334b1b0eed52197f46',
                                              '02470cc05fe10488eae25b981d88bba34b3c80be'],
 '019bb48ca9ec40a0f221ed451ee9de2383f12538': ['e995a9ec8d260fa063a244c50acdea719c81164f',
                                              'f88d5f342913e4719714c774b9c48fc39171981

 'a0a63307264f6b7a5a33ab0d917e9940a62f3def': ['51b2e01e201a86e11d8b7230d3f6cc62b9288e45',
                                              'a9992440a9b817c771540ebde81e578108312cae',
                                              'b3b1fba0a98d71ca84cfb97e8e1975180ab66b34',
                                              'a2470cbd6d2197b1e2c4e8b1149c07582b33313d',
                                              '25e614b13a69492de317def4470d9e88542d9688',
                                              'e0b824999ff0955b7894a3495fa4ab78ee661e02',
                                              '8d3cb94605785461c7949c84d5aaba081047b55e',
                                              'dbe09c2e6c7fb461d9fe95bad7595abdb33ebc37',
                                              'af05b425514ea287b288d3768f9a4326810bcd4a',
                                              '7f53190df66ee7554e29ab27038d146e2c50c3aa',
                                              'ac66ef1e538e6946b9ff542a48e449484ad9f94d',
          

In [164]:
# Testing whether every author is associated with a paper

testing = set()

for paper in papers:
    for person in paper["authors"]:
        if person in unknown_authors or person in unknown_authors.values():
            testing.add(person)

# for person in testing:
#     pprint(coauthors[person])
#     print("meep")
# for person in testing:
#     pprint(coauthors[person])
#     print("boop")
    
# for person in unknown_authors:
#     if person in testing:
#         print("yes")
#     else: 
#         print("nooooooooooooooooooooooooooooooooooooooooooooooooooo")
        
# Lists all listed authors that are in both unknown and potential people i.e. coauthors the two share.
# Better method than what I used earlier
for person in testing:
    if person in unknown_authors and unknown_authors.values():
        print("yes")
        
# Apparently some authors don't feature in a paper? I'm not sure if this is intended. If so, I'm not sure why

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [173]:
coauthor_counts = []

# Iterates through unknown authors
for unknown, matches in unknown_authors.items():
    
    # Iterates through possible matches for each unknown
    for person in matches:
        
        # Checks if the unknown author is in a paper ???
        if unknown in coauthors:
            
            # Iterates through coauthors of unknown author
            for unk_coauthor in coauthors[unknown]:
                
                # CHecks to make sure that the potential match for the unknown is in the coauthors list 
                
                if person in coauthors and unk_coauthor in coauthors[person]:
                    pprint(coauthors[person])
                    print("yes")
                    
# Are there people in the database who are not associated with a paper?

['a2470cbd6d2197b1e2c4e8b1149c07582b33313d',
 '2b51cb151576f7551f121aea6fe752b27f4848ed',
 '19d8ed14be8663a055e2658747c49fd0c56294ca',
 'd8d44879084846682367341ecde573a1e944ccc7',
 '26ca99331013ba17a262de2d11aca87e6aead1fa',
 '2e4f7dcab8422265beaf566e795b5ada604c3933',
 'dbe09c2e6c7fb461d9fe95bad7595abdb33ebc37',
 '6702c999c18a2e48b4028844c459ad2ae79c8448',
 'af05b425514ea287b288d3768f9a4326810bcd4a',
 '7f53190df66ee7554e29ab27038d146e2c50c3aa',
 '8432e17fcfe2988f9fc2a411d060062086e3d506']
yes
['a2470cbd6d2197b1e2c4e8b1149c07582b33313d',
 '2b51cb151576f7551f121aea6fe752b27f4848ed',
 '19d8ed14be8663a055e2658747c49fd0c56294ca',
 'd8d44879084846682367341ecde573a1e944ccc7',
 '26ca99331013ba17a262de2d11aca87e6aead1fa',
 '2e4f7dcab8422265beaf566e795b5ada604c3933',
 'dbe09c2e6c7fb461d9fe95bad7595abdb33ebc37',
 '6702c999c18a2e48b4028844c459ad2ae79c8448',
 'af05b425514ea287b288d3768f9a4326810bcd4a',
 '7f53190df66ee7554e29ab27038d146e2c50c3aa',
 '8432e17fcfe2988f9fc2a411d060062086e3d506']
yes
['

In [ ]:
# if "75dc757852ded186f04bf13eae66c53803db5c82" in unknown_authors:
#     pprint(unknown_authors)

In [ ]:
# Test cell

for author_id, name in authors_ids.items():
    if "Ribeiro" in name and "Ana" in name:
        print(author_id)
print()
for paper in papers:
    if "8b80ee24d983f356b3117484ac1a6f9a2e7ce441" in paper["authors"]:
        print(paper["DOI"])

In [ ]:
# Test cell

for author_id, name in authors_ids.items():
    if "Krawiec" in name:
        print(author_id)
print()
for paper in papers:
    if "75dc757852ded186f04bf13eae66c53803db5c82" in paper["authors"]:
        print(paper["DOI"])